6# Предсказание молекулярных свойств на основе предобученной модели трансформера


Молекулы могут быть представлены в виде [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) нотации, которая является текстовыми строками. SMILES кодирует стрктуру и состав молекулы.

Текстовая модель может вытащить из SMILES скрытое представление молекулы, на основе которого можно делать предсказания свойств молекулы.

В данном ноутбуке мы попробуем предсказать растворимость и липофильность для молекул.


# Установка библиотек


In [1]:
# Grab Jaime's excellent condacolab package: https://github.com/jaimergp/condacolab
# Note: you should probably read the README file at that repo.
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
!mamba install -c conda-forge rdkit chembl_structure_pipeline


Выходные данные были обрезаны до нескольких последних строк (5000).
tornado                  [] (00m:01s) Waiting...
reportlab                [] (00m:00s) Waiting...
xorg-kbproto             [] (00m:00s) Waiting...
xorg-libsm               [] (00m:00s) Waiting...
xorg-renderproto         [] (00m:00s) Waiting...
xorg-libxext             [] (00m:00s) Waiting...
xorg-xproto              [] (00m:00s) Waiting...
boost-cpp                [] (00m:00s)      6 MB /     16 MB (  3.76 MB/s)
fontconfig               [] (00m:00s) Waiting...
lcms2                    [] (00m:00s) Waiting...
gettext                  [] (00m:00s) Waiting...
libpng                   [] (00m:00s) Waiting...
libgfortran5             [] (00m:00s) Waiting...
chembl_structure_pipelin [] (00m:00s) Waiting...
python-dateutil          [] (00m:00s) Waiting...
pillow                   [] (00m:00s) Waiting...
libxcb                   [] (00m:00s) Waiting...
xorg-libice              [] (00m:00s) Waiting...
sqlalchemy               

In [3]:
!git clone https://github.com/DSPsleeporg/smiles-transformer.git

Cloning into 'smiles-transformer'...
remote: Enumerating objects: 623, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 623 (delta 11), reused 6 (delta 2), pack-reused 599
Receiving objects: 100% (623/623), 72.26 MiB | 20.99 MiB/s, done.
Resolving deltas: 100% (419/419), done.


# Импорт библиотек

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import DrawingOptions

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.neighbors import NearestNeighbors

sys.path.append('./smiles-transformer/smiles_transformer')

import torch
from pretrain_trfm import TrfmSeq2seq
from build_vocab import WordVocab
from utils import split


from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score, precision_recall_curve, auc

# Utils

Создадим функции для обработки данных

In [5]:
# Функция принимает на вход один SMILES, укорачивает их
# При помощи словаря переводит символы в набор цифр
# Возвращает массив чисел
def get_inputs(sm):
    seq_len = 220
    sm = sm.split()
    if len(sm)>218:
        print('SMILES is too long ({:d})'.format(len(sm)))
        sm = sm[:109]+sm[-109:]
    ids = [vocab.stoi.get(token, unk_index) for token in sm]
    ids = [sos_index] + ids + [eos_index]
    seg = [1]*len(ids)
    padding = [pad_index]*(seq_len - len(ids))
    ids.extend(padding), seg.extend(padding)
    return ids

# Функция принимает на вход набор SMILES
# И применяет функцию get_inputs()
def get_array(smiles):
    x_id = []
    for sm in smiles:
        a = get_inputs(sm)
        x_id.append(a)
        
    return torch.tensor(x_id)

In [6]:
# функция отрисовки красивых молекул
def plot_mols(mols, unit=200, w=120, h=200, fontsize=1.0):
    drawer = Draw.MolDraw2DSVG(4*unit, 3*unit, w, h)

    opt = drawer.drawOptions()
    opt.padding = 0.1
    opt.legendFontSize = 20


    xs = np.array([0,1,2,3,0,1,2,3,0,1,2,3])*unit
    ys = np.array([0,0,0,0,1,1,1,1,2,2,2,2])*unit
    for i, (mol, x, y) in enumerate(zip(mols,xs,ys)):

        drawer.SetOffset(int(x), int(y))
        drawer.SetFontSize(fontsize)

        AllChem.Compute2DCoords(mol)
        Chem.Kekulize(mol)

        drawer.DrawMolecule(mol, legend=str(i))



    drawer.FinishDrawing()
    return drawer

# Предобученная модель

https://arxiv.org/pdf/1911.04738.pdf


Измерить в лаборатории молекулярные свойста для большого числа молекул - дорогой по времени и материалам процесс.



Что же делать?

Есть датасеты известных молекул состоящие из миллионов объектов. Один из таких корпусов - [ChemBL database](https://www.ebi.ac.uk/chembl/).

Что если мы попробуем вытащить скрытые свойства молекул имея на руках только SMILES, не используя предсказания растворимости?


Это делается при помощи моделей автоэнкодеров.

![](https://sun9-69.userapi.com/impg/apo1uut5kWtKKeuCJHQT49z4VrcyZLUwH-yPXw/Q7V_OFqiy0Y.jpg?size=1406x976&quality=96&sign=48e376482628d4fd8f61472253d40992&type=album)

Автоэнкодер принимает на вход молекулу, после серии преобразований сворачивает ее в вектор небольшой размерности, затем пытается из этого вектора предсказать исходное представление молекулы!

Если у него это выходит - значит, "скрытое" представление содержит достаточно признаков, которые отвечают за представление молекулы.

Теперь можно взять из обученной модели только ту часть, которая вытаскивает свойства молекулы (энкодер) и на этих векторах сделать свои предсказания!

## Transformer

В моделях, работающих с естественным языком SOTA моделями являются трансформеры.

Трансформеры включают в себя слои Attention - это слои, которые сохраняют дополнительную информацию о слове, которое они обрабатывают. Когда мы читаем предложения в тексте - мы обращаем внимание на конкретные слова. Attention реализуют внутри себя эту функцию. Передавая "важность" слова между слоями мы можем улучшить наши предсказания.

![](https://miro.medium.com/max/1200/1*KD1xANybFo4EC2V2unn3RQ.gif)

Трансформер состоит из нескольких блоков, включающих слои Attention.

![](https://miro.medium.com/max/1400/1*V2435M1u0tiSOz4nRBfl4g.png)

# План 

1. Загрузим предобученную модель трансформера
2. Вытащим вектора признаков молекул
3. Построим предсказания
4. Визуализируем полученные вектора

# Загрузка модели

10 баллов

In [7]:
# Загружаем словарь символы-номера символов
!wget https://www.dropbox.com/s/3m3z7jij5jcmyem/vocab.pkl

--2021-05-17 11:06:44--  https://www.dropbox.com/s/3m3z7jij5jcmyem/vocab.pkl
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/3m3z7jij5jcmyem/vocab.pkl [following]
--2021-05-17 11:06:44--  https://www.dropbox.com/s/raw/3m3z7jij5jcmyem/vocab.pkl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucef21cbe2937c5970d4a6381383.dl.dropboxusercontent.com/cd/0/inline/BOo4Sx5S95kdSGFXsmCJwLGsuae0bAbkhAzhcH7wClph5kUeIqv7GYndqMr2A-lq40v5exjGE-4IagVDm-yd4tFaNUWvggrEAeSY1jRlw31qjGtjgJBXm2bTzXHu56vEUfoGwPXkbdR7lrVmFMUgqJdv/file# [following]
--2021-05-17 11:06:44--  https://ucef21cbe2937c5970d4a6381383.dl.dropboxusercontent.com/cd/0/inline/BOo4Sx5S95kdSGFXsmCJwLGsuae0bAbkhAzhcH7wClph5kUeIqv7GYndqMr2A-lq40v5exjGE-4IagVDm-yd4tFaNUWvggrE

In [8]:
# загрузим веса предобученной модели
!wget https://www.dropbox.com/s/0c0ogcvfuthystj/trfm_12_23000.pkl

--2021-05-17 11:06:49--  https://www.dropbox.com/s/0c0ogcvfuthystj/trfm_12_23000.pkl
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0c0ogcvfuthystj/trfm_12_23000.pkl [following]
--2021-05-17 11:06:50--  https://www.dropbox.com/s/raw/0c0ogcvfuthystj/trfm_12_23000.pkl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfd77712b55fbd96865c21cc58b.dl.dropboxusercontent.com/cd/0/inline/BOqOdgVuq4g63UX1PSiKdsYUROqyRdoNdMmcXRJjsTFTOia-6qTrQpTf2dfUqe3z1DD4FWQnIV3mgdGUBsH5CDsr9wMQIUdEo6FJv9yB0WdYsCxyaHi5H-nP3p60Yd7AdzXD9IYP1-l19pgjlURanyb0/file# [following]
--2021-05-17 11:06:50--  https://ucfd77712b55fbd96865c21cc58b.dl.dropboxusercontent.com/cd/0/inline/BOqOdgVuq4g63UX1PSiKdsYUROqyRdoNdMmcXRJjsTFTOia-6qTrQpTf2dfUqe3z1DD4FWQn

In [9]:
pad_index = 0
unk_index = 1
eos_index = 2
sos_index = 3
mask_index = 4

In [10]:
vocab = WordVocab.load_vocab('vocab.pkl')

Теперь создадим модель трансформера


In [11]:
trfm = TrfmSeq2seq(len(vocab), 256, len(vocab), 4)
trfm.load_state_dict(torch.load('trfm_12_23000.pkl'))
trfm.eval()
print('Total parameters:', sum(p.numel() for p in trfm.parameters()))

Total parameters: 4245037


Загрузим наши данные

In [12]:
!wget https://www.dropbox.com/s/5b05tivi01a43np/delaney-processed.csv

--2021-05-17 11:07:20--  https://www.dropbox.com/s/5b05tivi01a43np/delaney-processed.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5b05tivi01a43np/delaney-processed.csv [following]
--2021-05-17 11:07:21--  https://www.dropbox.com/s/raw/5b05tivi01a43np/delaney-processed.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc28db3194d4ab350402282eb9f.dl.dropboxusercontent.com/cd/0/inline/BOpbhl72TTy9VPYMXEqxI8Uzpi3EwXyEQIVm3I3P6UQhhj1Fi5au07Z2OxI1Y-Fllop2fJRbQ6TLOgYoB5S-MhJ1F7fMC1gRLF88aFDCGKHaZbm8BiXM239lMmvmNVGcQPP-_ySYbMc62g380o7XDpYh/file# [following]
--2021-05-17 11:07:21--  https://ucc28db3194d4ab350402282eb9f.dl.dropboxusercontent.com/cd/0/inline/BOpbhl72TTy9VPYMXEqxI8Uzpi3EwXyEQIVm3I3P6UQhhj1Fi5au07Z2OxI1

In [13]:
data = pd.read_csv('delaney-processed.csv')

In [14]:
x_split = [split(sm) for sm in data['smiles'].values]
xid = get_array(x_split)

Имея на руках SMILES, переведенные в наборы цифр, можем получить свойства молекул из нашей модели.

In [15]:
X_esol = trfm.encode(torch.t(xid))
print(X_esol.shape)

There are 1128 molecules. It will take a little time.
(1128, 1024)


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [17]:
Pipeline

sklearn.pipeline.Pipeline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import pipeline
from sklearn.preprocessing import StandardScaler

def evaluate_regression(X, y, model='ridge'):
    if model=='ridge':
        reg = Ridge()
    elif model=='rf':
        reg = RandomForestRegressor(n_estimators=10)
    else:
        raise ValueError('Model "{}" is invalid. Specify "ridge" or "rf".'.format(model))

    # разделите данные на тренировочную и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y) 

    # примените модель reg к тренировочным данным

    reg.fit(X_train, X_test)

    # сделайте предсказания на тестовой выборке
    y_pred = #your_code

    # посчитайте rmse и r2 для предсказаний
    r2 = #your_code
    rmse =  #your_code
    ret = {}
    ret['r2'] = r2

    ret['rmse'] = rmse

    return ret

In [ ]:
score_dic_esol = evaluate_regression(X_esol, data['measured log solubility in mols per litre'].values, model='ridge')
print(score_dic_esol)

Теперь давайте попробуем применить этот подхрод к датасету Lipophilicity!

In [19]:
!wget https://www.dropbox.com/s/dsrl00tj4zjmqbz/Lipophilicity.csv


--2021-05-17 11:08:53--  https://www.dropbox.com/s/dsrl00tj4zjmqbz/Lipophilicity.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/dsrl00tj4zjmqbz/Lipophilicity.csv [following]
--2021-05-17 11:08:53--  https://www.dropbox.com/s/raw/dsrl00tj4zjmqbz/Lipophilicity.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc488d7a68da2f7142f20583a199.dl.dropboxusercontent.com/cd/0/inline/BOrqhCwzxh2fjnVEz-19mS5m8xjYLApTJNY_DxU_mZuvd72c9vKAqlJHDCOYsCPfIDpdHH1a9ZBzm0t4x_qMC97vOaEdnSFvZot9085q3cWiT-kDPXsM8lnBVXG3iQPMY0UAhS3mLXscC5HJUXKGsPRx/file# [following]
--2021-05-17 11:08:54--  https://uc488d7a68da2f7142f20583a199.dl.dropboxusercontent.com/cd/0/inline/BOrqhCwzxh2fjnVEz-19mS5m8xjYLApTJNY_DxU_mZuvd72c9vKAqlJHDCOYsCPfIDpdHH1a

**TO-DO**

1. Закодируйте smiles датасета в числовые последовательности
2. Примените модель трансформера для получения векторов свойств молекул
3. Посчитайте модель регрессии на полученных векторах

In [ ]:
data_lip = #your_code
x_split_lip = #your_code
xid_lip = #your_code

In [ ]:
X_lip = #your_code
print(X_lip.shape)

In [ ]:
score_dic_lip = evaluate_regression(X_lip, data_lip['exp'].values, model='ridge')
print(score_dic_lip)

 Ой-ой...
 Что не так?

 Интерпретация моделей глубокого обучения - сложная тема. Существует множество методов, как понять, узнала ли нейросеть что-то из данных.

 Давайте попробуем нарисовать свойства, которые модель получила для наших датасетов и сравним.

# Визуализация

10 баллов

Для отрисовки векторов большого размера на плоскости мы будем использовать функцию tsne. Эта функция пытается сопоставить каждому вектору большого пространства вектор на плоскости так, чтобы близкие вектора из большого пространства были близкими на плоскости - и наоборот.

In [20]:
from sklearn.manifold import TSNE

 **TO-DO**

 1. создайте объект TSNE. В качестве параметров передайте размерность пространства, которую мы хотим получить на выходе.
 2. Примените метод fit_transform к векторам, полученным из трансформера.
 3. Вытащите целевую переменную
 

In [ ]:
ts = #your_code
X_esol_reduced = #your_code
y_esol = #your_code

Отрисуем полученные вектора

In [ ]:
fig = plt.figure(figsize=(12,10))
plt.rcParams['font.size'] = 12
plt.scatter(X_esol_reduced[:, 0], X_esol_reduced[:, 1], c=y_esol, marker='o')
plt.colorbar()
plt.show()

**TO-DO**

Сделайте визуализацию для датасета Lipophilicity

In [ ]:
ts = #your_code
X_lip_reduced = #your_code
y_lip = #your_code

In [ ]:
fig = plt.figure(figsize=(12,10))
plt.rcParams['font.size'] = 12
plt.scatter(X_lip_reduced[:, 0], X_lip_reduced[:, 1], c=y_lip, marker='o')
plt.colorbar()
plt.show()

**TO-DO:**

Что можно сказать о свойствах векторов двух датасетов? Почему предсказания на векторах из датасета липофильности такие плохие?

Ваш ответ

Посмотрим, что за молекулы нарисованы на плоскости.

Выберем последовательность находящихся рядом молекул.
Для этого используем модель, которая подбирает ближайших соседей к точке

**TO-DO**:
1. Создайте объект NearestNeighbors, передайте аргумент metric='euclidean' (считаем расстояние по как евклидово)
2. Примените модель к X_esol_reduced (fit)
3. Задайте область, в которой будем искать соседей при помощи np.linspace (на вход принимает начало области по оси, конец области и количество точек). Размер области посмотрите по картинке с отрисованными векторами.

In [ ]:
nn = #your_code
nn.#your_code

xs = np.linspace(#your_code)
ys = np.linspace(#your_code)
ids = []
pts = []
for x,y in zip(xs, ys):
    _, result = nn.kneighbors([[x, y]], n_neighbors=1)
    ids.append(result[0, 0])
    pts.append(X_esol_reduced[result[0, 0]])
pts = np.array(pts)

In [ ]:

fig = plt.figure(figsize=(8,7))
plt.rcParams['font.size'] = 14
plt.axis('off')
plt.scatter(X_esol_reduced[:, 0], X_esol_reduced[:, 1], c=data['measured log solubility in mols per litre'].values, marker='o')
for i in range(12):
    plt.scatter(pts[i,0], pts[i,1], c='r', marker='${}$'.format(i), s=200*(1+i//10))
plt.colorbar()
plt.savefig('esol.png', dpi=300)
plt.show()

In [ ]:
from IPython.display import SVG
mols = [Chem.MolFromSmiles(sm) for sm in data['smiles'].values[ids]]
dr = plot_mols(mols, w = 50)
with open('esol.svg', 'w') as f:
    f.write(dr.GetDrawingText())
SVG(dr.GetDrawingText())

Что вы видите на получившейся картинке?

*Ваш ответ*